# Understanding the Cloud through data analysis
This project is accesible at: https://github.com/rita-imdea/ripe-userguide 
The supplementary PDF file can be found at https://github.com/rita-imdea/ripe-userguide/blob/main/latency2_cloudDataAnalysis_GUIDE_student.pdf

In this notebook, we are going to see how to retrieve measurements fron RIPE Atlas and also how to parse measurements from JSON files (which have been pre-downloaded from RIPE Atlas). 
We recommend the reader to first take a look at the PDF document that contains the guide for the whole course. 

------------------------------

This notebook is divided in 2 parts. 

    
#### A) PARSING YOUR MEASUREMENTS
    We will explain how to clean and pre-process the data, which are *essential* steps to obtain good results.
    
#### B) ANALYZING YOUR MEASUREMENTS 
    We show how important is to understand the data before drawing some conclusions. After that, we apply an statistical analysis, where we will see both simple statistical models and machine learning models based on training.
    
  
------------------------------

This project makes use of data from RIPE Atlas, a measurement platform that contains public measurements. These measurements consists of the parameters and statististics of all the messages sent from some source devices, named "probes", towards some IP addresses or urls, named "destinations". These data allow us to understand how Internet works and how time and space impact on latency. 

Let's start!

------------------------------
------------------------------

First of all, we need to install some python packages that are required to contact with the RIPE Atlas server and to parse JSON files. Some other packages will be imported later on when they are needed. Make sure that you have installed the packages (see the guide that is accessible from the link on top of this file)

In [ ]:
import requests # to create http requests from Python
import json     # Library to write and ready JSON files in Python
from matplotlib import pyplot as plt # To plot the results

Next, we check how to pass from a retrieved JSON file to a meaningful format in Python to work with the data.


### A) PARSING YOUR MEASUREMENTS

We are going to get the data from the JSON files, which are already present in the course directory; you can also obtain other measurements from https://atlas.ripe.net/measurements/, as explained in https://github.com/rita-imdea/ripe-userguide/blob/main/latency1_whereIsTheCloud_B_getMeasurement.ipynb

The next steps are:

#### a) Initially, we parse the JSON files and store the data in a Python structure 
#### b) We clean the data, add new information, remove useless fields, and transform the data to provide the correct format to the algorithms. 

-----------------------------------------


First, we set the IDs of the measurements we are interested in. 
In this case, the IDs provided below refer to the measurements taken for this course. 

In [ ]:
#measurement_ids = ["48819905", "48819907", "48819909"] # example of measurement ID -- PING MEASUREMENTS
#measurement_ids = ["48819906", "48819908", "48819910"] # example of measurement ID -- TRACEROUTE MEASUREMENTS
measurement_ids = ["48819905",  "48819907",  "48819909"] # example of measurement ID


### a) Reading JSON files in Python

Next, we use the Python library "json", which allows us to ready and write JSON files with Python.
Furthermore, we use two of the most used packages for data processing in PYthon: Pandas and Numpy. 

1. Pandas is a Python library that provides a data structure called a DataFrame (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), which is a structure that facilitates data processing and manipulation. We can visualize it as a matrix that contains data of different format. We shall use it to parse our data because it provides a number of useful functions for manipulation and visualization of data.

2. Numpy is a Python library that facilitates mathematical operations, in particular for arrays and matrices. 

3. We also import datetime to work with different time formats.

In [ ]:
# We first import the necessary libraries 
import pandas as pd
import numpy  as np
from datetime import datetime

For each one of the measurements, we read the JSON file and store it in a pandas DataFrame (df) structure.

Then, we create a list of DataFrames (df) with all the measurements. 

In [ ]:
# Loop through the measurement IDs experiment files and create a DataFrame for each

dfs = [] # creating an empty vector for DataFrame elements

for measurement_id in measurement_ids: # For each of the measurements that we are considering:
    
    # Read the JSON data from the file
    with open(f"RIPE-Atlas-measurement-{measurement_id}.json", "r") as f:
        json_data = json.load(f)
    
    # Normalize the JSON data into a pandas DataFrame
    df = pd.json_normalize(json_data)
    
    # Append the DataFrame to the list of DataFrames
    dfs.append(df)


We would like to have all the DataFrames together to analyze all the measurements at the same time. Thus, we concatenate the list of dataframes.

In [ ]:
# Concatenate all the DataFrames into a single one
result_df = pd.concat(dfs, ignore_index=True)

We can visualize the current data structure thanks to the pandas function "head", which prints the first 5 rows of the DataFrame with the value of all the columns.

In [ ]:
# Print the resulting DataFrame
result_df.head()

### b) Data cleaning and pre-processing

The following step is to clean the data collection. 

For that, we should first know that the RIPE Atlas measurements may include negative values for latency (-1.0), which represent samples for which it was not possible to store the value (Due to, for example, timeouts in the network protocols). 


To avoid that these values impact the later analysis, we remove those samples. We first transform the negative values into Not-a-Number values (NaN), and we make use of the function "dropna" from Pandas library to remove those samples.  

In [ ]:
## Cleaning the data 

original_data_size = result_df.shape # saving the number of rows and columns for later

# Change the negative values to NaN
result_df['avg'].replace(-1.0, np.nan, inplace=True)

# Remove NaN values 
result_df = result_df.dropna(how='any',axis=0) 

print(f"Original size: {original_data_size} /// Processed size = {result_df.shape}")


We can easily check the percentage of "broken" samples that there were:

In [ ]:


print(f"We have {original_data_size[0]-result_df.shape[0]} bad samples {original_data_size[0]} among total samples.")

print(f"That means that {(original_data_size[0]-result_df.shape[0])/original_data_size[0]*100:.2f}% of the samples have been discarded")


We further pre-process the data to facilitate the plotting, visualization and readability of the data.

For that, we change the probe ID with an acronym that represents the country where the probe is located, plus an index in case there are several probes in the same country. 

In [ ]:
# Renaming the probe_id column for easy plotting 
nprb_id = []

for value in result_df["prb_id"]:
    if value == 1004991:
        nprb_id.append('es1')
    if value == 53229:
        nprb_id.append('es2')
    if value == 1004997:
        nprb_id.append("pt1")
    if value == 1004102:
        nprb_id.append("pt2")
    if value == 20757:
        nprb_id.append("es3")
    if value == 1003454:
        nprb_id.append("nl1")
    if value == 1003158:
        nprb_id.append("nl2")
    if value == 1003747:
        nprb_id.append("nl3")
    if value == 1004200:
        nprb_id.append("es4")
    if value == 51381:
        nprb_id.append("ug1")
    if value == 54470:
        nprb_id.append("us1")
    if value == 1002914:
        nprb_id.append("us2")
        
# Saving the new IDs in the column "nprb_id"
result_df["nprb_id"] = nprb_id


Now, we edit the format in which the time is saved. By default, the time units provided by RIPE Atlas are UNIX timestamps, i.e., the number of seconds passed since 1st January 1970 at 00:00h.

In [ ]:
## Changing the time column from epoch to date time format for time series processing

new_timestamp = [] # initializing the new array

# populating the new array
for i in result_df['timestamp']:
    my_datetime = datetime.fromtimestamp(i) # transforming the timestamp in standard time
    new_timestamp.append(my_datetime) # adding the value to the array

# Saving the new time format in the column "new_time"
result_df['new_time'] = new_timestamp
result_df['new_time']

The last pre-processing is to save the code of the country in which the probe is located. 

In [ ]:
# itializing the country code array
country_name = []

for value in result_df["nprb_id"]:
    if (value == 'es1') or (value == 'es2') or(value == 'es3') or( value == 'es4'):
        country_name.append('Spain')
    if(value == 'nl1') or (value == 'nl2')or (value == 'nl3'):
        country_name.append('Netherlands')
    if(value == 'pt1') or (value == 'pt2'):
        country_name.append('Portugal')
    if(value == 'us1') or (value == 'us2'):
        country_name.append('USA')
    if(value == 'ug1'):
        country_name.append('Uganda')

# Saving the new country code in the column "country_name"
result_df['country_name'] = country_name


We can visualize if the new additions are correct by calling again the method df.head(). You will see that the last three columns coincide with the content that we have just added.

In [ ]:
result_df.head()

------------------------------

We have already cleaned and structured our data. Now, it's time to analyze and extract valuable information from it. 

------------------------------
------------------------------
------------------------------

### B) ANALYZING YOUR MEASUREMENTS 

We split this section also in two parts. 

a) Check how latency varies over time, how mean and standard deviation vary over distance or any other interesting scenarios you can come up with 

b) Finally we can do some predictions based using established mathematical models or machine learning models and see what gives us the best results.  

### a) Understanding the data
The first step is to take a look at the data we have and try to obtain insights that can help us with the later analysis.

Let us first check what is the probability distribution of the latency for the whole dataset.

For that, we plot the discrete PDF based on the histogram of the data.

In [ ]:
# Computing the discrete PDF from the histogram

axx = result_df['avg'].hist(density=True, bins=100, alpha = 0.8) 
# bins = Number of points in the histogram.
# alpha = a parameter for color transparency

# We can add information into the graph
axx.set_ylabel("Probability Density Function")
axx.set_xlabel("Latency (ms)")

We realize that the shape is quite curious, with several peaks, looking like a decreasing wave. 

**What can be the reason?**<br>

**Please, take a moment to think about it and comment in group what you think about the graph above**<br> 

Don't continue further down until you took your time to talk about it.

---------------------------------------------------------------

**... 
<br><br>
...
<br><br>
...
<br><br>
...
<br><br>
...
<br><br>
...
<br><br>**

Have you already thought about it? Let us see if we can dive a bit deeper into the data.

Let us consider now some of the information we have to try to get more insights.<br>

First, let us check what is the PDF for the data originated in each probe, separately.


In [ ]:
## checking PDF for each of the probes

# First, we recover the array of probe IDs. These IDs are identifiers used for RIPE Atlas, the measurements platform. 
probes = [1004991,53229,20757,1004200,1003454,1003747,1004102,51381,54470,1002914]
         
# We extract the data corresponding to each probe, and plot it separately but in the same plot
for probe in probes:
    df_cdf = result_df[(result_df['prb_id'] == probe)]  # Get the data that corresponds to one probe
    axx = df_cdf['avg'].hist(density=True, bins=100, alpha = 0.3) # Draw the histogram

# We add the probe IDs as legend
axx.legend(probes)
axx.set_xlabel("Latency (ms)")

What can we observe?<br>

**Any ideas??**<br>


Take a moment to propose ideas before continuing. 
<br>

---------------------------------------------------------------

**... 
<br><br>
...
<br><br>
...
<br><br>
...
<br><br>
...
<br><br>
...
<br><br>**

(...)


 
<br><br>

<br><br>

<br><br>

<br><br>

<br><br>

<br><br>


So, it seems that there are some probes that are easily differentiable from the rest of the probes.
But that plot shows us little information about **why**.

Also, the probe IDs is not a very meaningful value. We can use the attibute that we have previously added identifying probes with the country where they are located. Let us know plot the same graph with the new ID.

In [ ]:
## checking PDF for each of the probes

# First, we recover the array of probe IDs
probes = ['es1','es2','es3','es4','nl1','nl3','pt2','ug1','us1','us2']
         
# We extract the data corresponding to each probe, and plot it separately but in the same plot
for probe in probes:
    df_cdf = result_df[(result_df['nprb_id'] == probe)] # Get the data that corresponds to one probe
    axx = df_cdf['avg'].hist(density=True, bins=100, alpha = 0.3) # Draw the histogram

# We add the probe IDs as legend
axx.legend(probes)
axx.set_xlabel("Latency (ms)")

It is clear that the country of location have an important impact. In order to better verify it, we can show the results aggregated by country. Let us see what is the outcome of this visualization.

In [ ]:
## Obtaining PDF per country

# Retrieving list of countries from the data
countries = result_df['country_name'].unique()

for country in countries:
    df_pdf = result_df[(result_df['country_name'] == country)] # Get the data that corresponds to one country
    axx = df_pdf['avg'].hist(density=True, bins=100, alpha = 0.3) # Draw the histogram 

axx.legend(countries)
axx.set_xlabel("Latency (ms)")

Now we see more clearly that **location matters** for latency and internet. This, which is expected by all of us, it is not necessary true if we do not corroborate it with data. 

More in detail, Portugal and Spain obtain the same performance, as they are next to each other and the servers are located in further places such us Netherlads, U.S., and other countries. 

This results showcases the fact that, if we want to know whether some edge-cutting application can be deployed in a certain place, we need to first make sure that the network has a sufficiently good performance to actually allow for the service to work.

In the following, we advance a bit deeper on the characterization of the data. 


-----------------------------------

One crucial value to understand the latency values is to know where each of the probes and destinations are located. 

For that, we now obtain the coordinates of all the devices (sources and destinations) participating in the experiment, and we include it in the DataFrame object. 

In [ ]:
## Obtaining geographical information of the probes 

# Import the package needed to format the metadata of RIPE's probes.
from ripe.atlas.cousteau import Probe 

#Getting all the ripe atlas probes
probe_id_list = result_df['prb_id'].unique()
probe_nid_list = result_df['nprb_id'].unique()

# Collect the source probe latitude/longitude information from ripe atlas 
probe_coordinates = []
probe_country = []

for id_i in probe_id_list: # for each one of the probes:
    probe = Probe(id=id_i) # Obtains all metadata of probe id_i
    #print(probe.geometry) #probe.geometry is a GeoJSON https://en.wikipedia.org/wiki/GeoJSON
    probe_coordinates.append(probe.geometry['coordinates']) # saving the coordinates to the list of coordinates
    probe_country.append(probe.country_code)



We now have a list with the information of coordinates for each of the probes. Next, we tranform it into latitude and longitude, and we add the country in which they are located. 

In [ ]:
longitude = []
latitude = []

for i in probe_coordinates:
    longitude.append(i[0])
    latitude.append(i[1])

# create a probe metadata dataframe
srcprobes_df = pd.DataFrame({'prb_id': probe_nid_list,'longitude': longitude, 'latitude': latitude,'probe_country': probe_country})
srcprobes_df

We have done this for the probes (source nodes). We can repeat the procedure for the destination addresses.

In [ ]:
## Obtaining geographical information of the destinations 

#Getting all the destination ip addresses
dest_ip_list = result_df['dst_addr'].unique()

# Create empty lists to store the data
server_ip = []
latitudes = []
longitudes = []

# Query the "ipinfo.io" API for geolocation information
for ip in dest_ip_list:
    response = requests.get(f'https://ipinfo.io/{ip}/json')
    if response.status_code == 200:
        data = response.json()
        latitude, longitude = data.get('loc', '').split(',')
        server_ip.append(ip)
        latitudes.append(latitude)
        longitudes.append(longitude)

# Create a dataframe with the geophraphical information of the destinatiuon addresses. Remember that this infomation is approximate for privacy reasons
a_server_df = pd.DataFrame({ 'Server_ip': server_ip,'latitude': latitudes, 'longitude': longitudes})
a_server_df

We have now all the information regarding the location of the addresses participating in the experiment. Thus, we can now calculate the distance between sources and destinations. 

In [ ]:
## Obtaining the distance between nodes. 

# We first define the function that computes the Euclidean distance between two points
distance_data = []
def euclidean_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

# Create an empty dataframe to store distance information
distance_df = pd.DataFrame(columns=['prb_id', 'ip_address', 'distance'])

# Calculate Euclidean distances between all probes and destination addresses
for index_probe, row_probe in srcprobes_df.iterrows():
    for index_dst, row_dst in a_server_df.iterrows():
        distance = euclidean_distance(float(row_probe['latitude']), float(row_probe['longitude']), float(row_dst['latitude']), float(row_dst['longitude']))
        distance_data.append({
            'nprb_id': row_probe['prb_id'],
            'dst_addr': row_dst['Server_ip'],
            'distance': distance})
        
# Create a DataFrame from the list of distance information
distance_df = pd.DataFrame(distance_data)


We can show the distance between all probes and destinations as follows: 

In [ ]:
distance_df

We now add this information to our general DataFrame.

In [ ]:
# Adding the distance data to the result_df.
# For that, for each of the points that have the same ['nprb_id', 'dst_addr'] tuple, we add the field of distance between those nodes. 
analysis_df  = pd.merge(result_df,distance_df, left_on=['nprb_id', 'dst_addr'], right_on=['nprb_id', 'dst_addr'], how='inner')


We can visualize the first 5 rows to check that the value is there. 

In [ ]:
analysis_df.head()

------------------------------------

### b) Predicting the data

Now it is turn to see whether we can actually predict the latency on Internet!


We are going to start with very simple forecasting methods, to observe if the data is easily predictable or we need more complex algorithms. After that, we will check also more involved machine learning algorithms. 

----------------------

#### Simple methods

----------------------

We start evaluating simple and classical methods that offer an efficient (but limited) manner of predicting the next values. 
These methods are: the *Naive forecast* and *Exponential Smoothing*. 

We aim at predicting the next values of the time series of average latency. The column in our dataset representing the average latency is 'avg'.

But... **there is a very important step to apply before predicting!!!**

These methods take the previous N samples to estimate the next one. For that, we need to structure or data such that it follows a strict time order. Note that if we check now our data:

In [ ]:
result_df[['src_addr', 'dst_addr', 'new_time']].head(15) # Print first 15 rows of some columns

***If we try to estimate the future sample of one link with the previous samples, we are using samples from other links!***
We need to correctly order the dataframe:

In [ ]:
### We sort the values by link ('src_addr', 'dst_addr') and time ('timestamp')
sorted_df = result_df.sort_values(by=['src_addr', 'dst_addr', 'timestamp'], inplace = False) # inplace=False == New object
sorted_df.reset_index(drop=True, inplace = True) # We reset the index of the dataframe to avoid issues.
sorted_df[['src_addr', 'dst_addr', 'new_time']].head(15) # Print first 15 rows of some columns

Let us now evaluate the two methods. 

The first one is as simple as "Naive forecast", which consists on just considering that the next sample is going to be exactly the same as the previous one. 

In [ ]:
# Applying some simple forecasting methods

# Using the naive forecast
sorted_df['naive'] = sorted_df['avg'].shift(1) # next sample is the previous one
sorted_df[['avg', 'naive']].head(7) # Print first rows of some columns

Of course, the first value do not have any forecast. To avoid problems with NaN, we assign the exact value for the first sample:

In [ ]:
# Replace NaN at top of value 
sorted_df.at[0,'naive'] = sorted_df.at[0,'avg']
sorted_df[['avg', 'naive']].head(7) # Print first  rows of 'avg' and 'naive'

We can evaluate the Mean Squared Error of this algorithm as follows.

In [ ]:
# Testing the prediction accuracy for naive forecast
se = (sorted_df['avg'] - sorted_df['naive']) ** 2
mse_naive = se.mean()
mse_naive

Let us compare this simple algorithm with more involved ones. 
The next one is Exponential smoothing. 


**Exponential smoothing** is a technique qhere the next sample is obtained as a weigthed moving average of previous samples. The weights are selected such that they decrease exponentially. 
The simplest form of exponential smoothing is given by the following expressions, where y_t is the prediction at time t for the future sample and x_t is the actual value at time t.

$y_0 = x_0$

$y_t = \alpha x_t + (1-\alpha)y_{t-1}$ for $t>0$

We can evaluate this algorithm:

In [ ]:
## Exponential smoothing method

# import the library that allows us to easily apply it
from statsmodels.tsa.api import SimpleExpSmoothing

# Fitting (adjusting) the data to an exponential smoothing approach. 
fit1 = SimpleExpSmoothing(sorted_df['avg']).fit()

# Evaluating the result of the fitting
sorted_df['simple_smoothing'] = SimpleExpSmoothing(sorted_df['avg']).fit().fittedvalues

As for the Naive forecast, we can evaluate the Mean Squared Error of this algorithm as follows.

In [ ]:
se = (sorted_df['avg'] - sorted_df['simple_smoothing']) ** 2
mse_esm = se.mean()

And we can compare them:    

In [ ]:
print(f"MSE Naive: {mse_naive:.2f}, MSE Exponential smoothing: {mse_esm:.2f}")

Exponential smoothing already improves w.r.t. the naive method.

We can plot the results of both methods as follows

In [ ]:
# Plotting the results
samp_start_plot = 300 # last sample showed
samp_end_plot   = 700 # fir sample showed

plt.plot(sorted_df['avg'].iloc[samp_start_plot:samp_end_plot],  linewidth = 2, color = 'black')
plt.plot(sorted_df['naive'].iloc[samp_start_plot:samp_end_plot], '.', color='green', linewidth=1)
plt.plot(sorted_df['simple_smoothing'].iloc[samp_start_plot:samp_end_plot], '.', color = 'blue', linewidth=1)
plt.title('Simple methods')

fig = plt.gcf()
fig.set_size_inches(10, 7)
fig.set_dpi(200)

plt.legend(['Real latency', 'Naive', 'Exponential smoothing'])
plt.xlabel("samples")
plt.ylabel("latency (ms)")

We can see how the naive method is more sensitive to noise and the exponential nature of the smoothing can be observed in the peaks: the following samples depend on the peak/outlier value in a way that decreases exponentially.

Now, we move to evaluating machine learning algorithms. 

We are going to begin with a simple problem that aims at predicting the distance based on the latency between source and destination. 

----------------------

#### Machine learning methods   

----------------------

The first step is to separate the data in training and test set. The training set is the data we will use to train the model and let it learn. Then, we evaluate the performance in the test data. This approach is needed in order to avoid that the method "overfit", that is, that it adapts too much to the data it sees but it is not able to adapt to new data. 

First, we import some useful packages

In [ ]:
## Using machine learning models

# We import the necessary packages. These ones are already included in the Python distribution, so you don't need to install them
import itertools
import random

Now we split the data as explained. For that, we split the data based on the source-destination pair. We select 70% of the pairs as training data and the remaining 30% as test data. 

In [ ]:
## First we randomize the dataset and then split it into training and testing sets before applying the models
df2 = analysis_df.copy() # Making a copy of the data

## First of all, we select a subset of source-destination pairs for training and the rest for testing the training.
test_indices  = []
train_indices = []
        
# We obtain the identifiers for source and destination
array1 = df2['nprb_id'].unique()
array2 = df2['dst_addr'].unique()

# Creating all possible pairs of source - destination
pairs = list(itertools.product(array1, array2))

# Randomly selecting the 30% of those pairs for testing
num_test_pairs = int(len(pairs)/3)
test_pairs     = random.sample(pairs, num_test_pairs)

# Creating separate lists
test_list  = test_pairs
training_list = pairs.copy()

# Removing selected pairs from the original list
for pair in test_list:
    training_list.remove(pair)
    

Once we have randomly selected the pairs, we create two different dataframes. 

In [ ]:
## Retrieving training data
train_dfs = []
for i, k in training_list:
    # Get the data from the dataframe corresponding to this given pair
    temp_df = df2.loc[(df2['nprb_id'] == i) & (df2['dst_addr'] == k)]
            
    # Append the pair data to the selected data
    train_dfs.append(temp_df)

# Concat all training pairs' data
train_df = pd.concat(train_dfs)
  
## Retrieving test data

test_dfs = []
for i, k in test_list:
    # Get the data from the dataframe corresponding to this given pair
    temp_df = df2.loc[(df2['nprb_id'] == i) & (df2['dst_addr'] == k)]
            
    # Append the pair data to the selected data
    test_dfs.append(temp_df)

test_df = pd.concat(test_dfs)
        

Then, we need to do a very important step: the normalization of input data. This step is very important because it impacts considerably in the performance of the algorithms. A correct normalization of the input data may make a previously useless approach into the most accurate one, since the algorithms are very sensitive to the range and variation of the data. 


The normalization we use is such that we obtain a variable whose mean is zero and variance is 1. 
Note that, since we can only train the method with the training data, we consider the training mean and training variance as the correct values, although that will create a (usually small) error with respect to the real mean and variance. 

In [ ]:
## Normalization of input data

#normalize the latency values 
train_df['normalized_latency'] = (train_df['avg'] - train_df['avg'].mean()) / train_df['avg'].std()
test_df['normalized_latency']  = (test_df['avg']  - train_df['avg'].mean()) / train_df['avg'].std()

# normalize the 'distance' column
train_df['normalized_distance'] = (train_df['distance'] - train_df['distance'].mean()) / train_df['distance'].std()
test_df['normalized_distance']  = (test_df['distance']  - train_df['distance'].mean()) / train_df['distance'].std()


Once the input data is correctly structured, we can start evaluating different algorithms. 


We start evaluating a <b>Decision Tree<b>.
    
For that, we first need to import the required libraries that allow us to implement the decision tree learning without re-implementing it again. 
    

In [ ]:
## Import the Machine learning libraries related to Decision trees
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

Next, we obtain the input data and the "labels" (correct output values) for train and test. Generally, X represents the input and Y the output. 


In [ ]:
# Select your features and target
X_train = train_df['normalized_latency'].values.reshape(-1,1)
y_train = train_df['normalized_distance'].values

X_test = test_df['normalized_latency'].values.reshape(-1,1)
y_test = test_df['normalized_distance'].values

We can train the decision tree. Since the value we want to predict (latency) is continuous, we need a regressor.
In the next lines, we fit the model to our training data, which consists on optimizing the model parameters to reduce the error of the prediction for the training data. 

In [ ]:
## Create a Decision Tree Regressor
model = DecisionTreeRegressor()
## Train the model
model.fit(X_train, y_train)

As we can see, it is very simple with the available libraries. 
We can also verify the performance of this decision tree

In [ ]:
## Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the accuracy of the model
mse_dt = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error of the decision tree: {mse_dt:.2f}")

Note that the MSE above is normalized because the inputs are normalized. We can denormalize it to obtain:

In [ ]:
print(f"Mean Squared Error of the decision tree: {mse_dt*train_df['distance'].std():.2f} km")

It is a bit difficult to understand what is going on behind the code above. 

<b>We can also visualize the tree!<b>

In [ ]:
# Viewing the Decision Tree
from sklearn.tree import export_graphviz
import graphviz

## Generating the scheme of the tree based on the distance, with some extra parameters
tree_dot = export_graphviz(model,feature_names =["latency"],out_file=None, rounded=True, filled=True)

# Visualize the tree using Graphviz
graph = graphviz.Source(tree_dot)
graph

It will probably appear too big for the window. Don't worry, you can save the image to better understand it. 
It is quite simple: The model consists of a series of True/False questions, each "leaf" (end node) leads you to a predicted value.


After seeing the decision tree solution, we start evaluating a <b>LSTM Model<b>.

Long short-term memory networks are recurrent neural networks (RNN) that are a very good fit to time-series forecasting, as their output depends on past decisions.
    
For that, we also need to import the required libraries that allow us to implement the method.

In [ ]:
# LSTM Modelling 
# If you want to know more about RNN & LSTM, you can follow this 20 minutes tutorial: https://www.youtube.com/watch?v=BSpXCRTOLJA
import tensorflow as tf
from tensorflow. keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# We are going to use also one function from numpy package
from numpy import array


Importantly, RNN receive as input <b> Sequences <b>. This means that we have to re-structure the data such that each one of the input sets are time-ordered sequences. For that, we define the following function.

This function, "split_sequence", receives a time series and outputs the last "n_steps" samples for each one of the time samples. 

In [ ]:
## Creating sequences of past samples. 

# n_steps = number of previous samples to consider

# since we need n_steps previous samples, the prediction will start after the first n_steps time slots.

def split_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)-n_steps):
        seq_x = sequence[i:i + n_steps]
        X.append(seq_x)
    
    # Important! Shuffling to help the method to learn different patterns
    shuffled_X = np.array(X)
    np.random.shuffle(shuffled_X)

    return shuffled_X


Now that we have the function defined, we can pass the train and test data through it

In [ ]:
n_pastSamples = 30 # Number of past values that are taken into account for predicting. 

Xtrain_seq = split_sequence(X_train, n_pastSamples)
Xtest_seq  = split_sequence(X_test,  n_pastSamples)

Ytrain_seq = y_train[n_pastSamples:] # Because we can only predict from sample n_pastSamples
Ytest_seq  = y_test[n_pastSamples:] # Because we can only predict from sample n_pastSamples

print(f"The new shape of our training data is: {Xtrain_seq.shape}, and the shape of the training labels is: {Ytrain_seq.shape}")
print(f"The new shape of our test data is: {Xtest_seq.shape}, and the shape of the test labels is: {Ytest_seq.shape}")

Now it is turn to define the neural network architecture and the learning parameters. 

Let us start with a simple architecture. 

In [ ]:
## define model
n_features   = 1  # number of input features; in our case, only one: distance
n_neurons    = 100 # number of neurons per layer

model = Sequential() # Tipe of model

# We add one LSTM layer. The activation function is the Rectified Linear function.
# We also need to add the correct shape of the input values
model.add(LSTM(n_neurons, activation='relu', input_shape=(n_pastSamples, n_features)))

# We add an output layer consisting on a single neuron that outputs the predicted value.
model.add(Dense(1))

# We select the Mean Squared Error as the loss function, and we use the standard adam optimizer.
model.compile(optimizer='adam', loss='mse')

Time to fit (train) the model!

The parameter "epochs" denotes the number of "rounds" we pass through the training set. Let's train the model:

In [ ]:
## fitting the model
num_epochs = 20 # number of cycles of training through the training data

# If you don't want to see the information about training, set verbose = 0
model.fit(Xtrain_seq, Ytrain_seq, epochs=num_epochs, verbose=1)

And we can predict the test labels!

In [ ]:
y_pred = model.predict(Xtest_seq, verbose=1)

And, of course, evaluate the MSE:

In [ ]:
mse_lstm = mean_squared_error(y_pred, Ytest_seq)

print(f"Mean Squared Error of the simple LSTM network: {mse_lstm:.2f} normalized MSE")
print(f"Mean Squared Error of the simple LSTM network: {mse_lstm*train_df['distance'].std():.2f} km")

The internal parameters of the neural networks may matter a lot. The previous network is a very simple minimal LSTM network. Let's define more complex one. 

In [ ]:
## define model
n_features   = 1   # number of input features; in our case, only one: distance
n_neurons    = 100  # number of neurons per layer
dropout_prob = 0.2 # Probability of dropping out neurons (it helps to generalize the training)

model = Sequential() # Tipe of model

# We add one LSTM layer. The activation function is the Rectified Linear function.
# We also need to add the correct shape of the input values
# return_sequences = True is needed if the following layer is LSTM because they require sequences
model.add(LSTM(n_neurons, activation='relu', input_shape=(n_pastSamples, n_features), return_sequences = True))
model.add(Dropout(dropout_prob)) # drop out of some values to improve generalization

# We add one LSTM layer. The activation function is the Rectified Linear function.
model.add(LSTM(n_neurons, activation='relu'))
model.add(Dropout(dropout_prob)) # drop out of some values to improve generalization

# We add a fully connected layer of 32 simple neurons. 
model.add(Dense(32, activation='relu'))
model.add(Dropout(dropout_prob))

# We add an output layer consisting on a single neuron that outputs the predicted value.
model.add(Dense(1))

# We select the Mean Squared Error as the loss function, and we use the standard adam optimizer.
model.compile(optimizer='adam', loss='mse')



We can train this new model and check the performance. 

In [ ]:
## fitting the model
num_epochs = 20 # number of cycles of training through the training data

# If you don't want to see the information about training, set verbose = 0
model.fit(Xtrain_seq, Ytrain_seq, epochs=num_epochs, verbose=1)

y_pred = model.predict(Xtest_seq, verbose=1)

mse_lstm2 = mean_squared_error(y_pred, Ytest_seq)



In [ ]:
print(f"Mean Squared Error of the simple network: {mse_lstm2:.2f} normalized MSE")
print(f"Mean Squared Error of the decision tree: {mse_lstm2*train_df['distance'].std():.2f} km")